-----
Jaciane de Oliveira Cruz  - 118110412

----

Busca com K-NN




In [ ]:
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

Lendo os dados.

In [173]:
from IPython.core.display import display_pdf
url = "https://raw.githubusercontent.com/caiolibanio/atividade_NLP/master/csv_satiras_politicas.csv"
df = pd.read_csv(url)
#df.head()
pd.read_csv(url)



,title,text,label
0,crise e tao grande que nem tiozao do pave fez piada na noite de natal,a familia guimaraes passou a noite de natal perplexa. genival o tiozao do pave deles desde 1985...,satire
1,nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble,uma menina de 11 anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de j...,satire
2,marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e...,insatisfeitas com seus partidos com as siglas dos partidos com os politicos com as tardes ensola...,satire
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da america latina fez uma pesquisa que mostrou ideias que ...,satire
4,apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz,assassinatos sequestros mortes violentas. nenhuma noticia parece abalar a apresentadora do cida...,satire
...,...,...,...
4369,pesquisa 85 das pessoas ja abandonaram a dieta iniciada no novo ano novo,ferias de janeiro acabaram carnaval acabou. o novo ano novo iniciado na segundafeira poscarnava...,satire
4370,19h de sexta ja e hora de admitir a derrota e ligar para o peguete de sempre diz estudo,sextafeira dia de esperancas no coracao dos solteiros. mas tudo tem limite. pelo menos e o que...,satire
4371,senado aprova hoje nova reforma ortografica com uso do sinal de acentuacao de ironi,a lingua portuguesa passara a ser a primeira do mundo a ter acentuacao para a ironia. o senado ...,satire
4372,haddad prova que unica coisa que o pt nao rouba e voto dos adversarios,o prefeito fernando haddad teve que manter a velocidade baixa e nao conseguiu arrancar na reta f...,satire


Função para gerar matrizes TF-IDF a partir da coleção de documentos textuais passados como entrada.

In [ ]:
def buildTFIDFVectorizer(docs):
    vect = TfidfVectorizer()
    matrix = vect.fit_transform(docs) 
    return matrix, vect

Função para calcular o produto escalar entre uma matriz e um vetor (nesse caso o título de entrada). Como o vetor nesse caso será um vetor linha, precisamos transpor o vetor (para um vetor coluna) para que as dimensões da matriz e do vetor casem e a multiplicação possa ser realizada. 

Como resultado, teremos um array com os resultados do produto escalar entre o vetor de entrada e todos os outros (inclusive ele próprio).

In [ ]:
def docSimilarities(doc, matrix):
  res = matrix.dot(doc.transpose())
  return res

-----
#Análise  dos títulos

1 - Retornar os 3 vizinhos mais próximos usando só os títulos. Ou seja, usando um título como entrada e os outros títulos para calcular as similaridades;

Como nossa análise é somente nos títulos, vamos retirar as outras colunas.

In [ ]:
df_titles = df.drop(columns=['text', 'label'])

# Como o texto é truncado, aqui aumentamos o tamanho do texto a ser mostrado.
pd.options.display.max_colwidth = 100
df_titles.head()

,title
0,crise e tao grande que nem tiozao do pave fez piada na noite de natal
1,nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble
2,marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e...
3,dez propostas que podem realmente mudar o brasil
4,apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz


In [ ]:
# Recuperando os textos da coluna title do dataframe
titles = df_titles['title']


In [ ]:
# Aqui calculamos a matriz de TF-IDF dos títulos e guardamos o objeto da classe TfidfVectorizer que foi usado para isso. 
tfidf_matrix, tfidf_vect = buildTFIDFVectorizer(titles)

Aqui escolheremos um título de entrada para a busca. Nesse exemplo, escolhemos : 
* "brasileiro ordenha vaca e planta cacau para economizar na pascoa"
* "ladrao assaltado depois de roubar farmacia sai da cadeia e vai ao japao ver o corinthians"
* "papai noel fica preso no caos aereo e pode nao chegar para o natal"

In [ ]:
query_doc1 = tfidf_matrix.getrow(24)
query_doc2 = tfidf_matrix.getrow(73)
query_doc3 = tfidf_matrix.getrow(96)


Agora calculamos o produto escalar entre 'query_doc's e todos os outros documentos.

In [ ]:
doc_sim1 = docSimilarities(query_doc1, tfidf_matrix)
doc_sim2 = docSimilarities(query_doc2, tfidf_matrix)
doc_sim3 = docSimilarities(query_doc3, tfidf_matrix)

Como um array esparso é retornado, geramos um array denso do numpy para usarmos a função argsort, que ordenará uma array de entrada de forma crescente e retornará os índices correspondentes a esses valores.

In [ ]:
doc_sim1 = np.array(doc_sim1.toarray())
doc_sim2 = np.array(doc_sim2.toarray())
doc_sim3 = np.array(doc_sim3.toarray())

Aqui retornamos o índice do documento que tem o maior valor de produto escalar (ou seja o documento mais similar ao documento de entrada). Note que como o próprio documento de entrada também é considerado no cálculo, usamos o índice do segundo maior valor. Como o documento é ordenado em ordem crescente, precisamos pegar o penúltimo valor (por isso o índice -2).

In [ ]:
mostSimilar1 = np.argsort(doc_sim1, axis=None)[-4:-1]
mostSimilar2 = np.argsort(doc_sim2, axis=None)[-4:-1]
mostSimilar3 = np.argsort(doc_sim3, axis=None)[-4:-1]


Os títulos mais similares ao titulo escolhido

In [176]:
df.iloc[mostSimilar1]

,title,text,label
1136,pra economizar passagem mulher tenta levar marido no compartimento de malas,final de ano confusao nos aeroportos passagens caras. elisangela nunes 23 anos e seu marido jef...,satire
1706,temer volta as pressas para economizar com hotel e cunha oferece cela em curitiba,michel temer resolveu adiantar sua volta do japao. a assessoria da presidencia nega que o motiv...,satire
4249,seca vaca de sao paulo produz leite em po,o pecuarista aureliano nascimento nao acreditou quando ordenhou hoje pela manha a vaca mimosa. ...,satire


In [177]:
df.iloc[mostSimilar2]

,title,text,label
139,depois de ver candidatos virus do ebola desistiu de vir ao brasil,suspeito de ter vindo passar uma temporada no brasil o virus do ebola nao confirmou presenca. a...,satire
111,embaixada do brasil no japao ja tem foto de temer na parede,a embaixada brasileira em toquio onde ja passa do meiodia de domingo ja conta com retratos ofici...,satire
4255,encontrado primeiro ladrao a desistir de roubar casa porque deram duas voltas na fechadura,finalmente foi encontrado o primeiro ladrao a desistir de arrombar e roubar uma casa apos os don...,satire


In [178]:
df.iloc[mostSimilar3]


,title,text,label
3715,papai noel confirma que dilma nao sabia de nada sobre a petrobras,o bom velhinho estava cansado. ainda assim ele resolveu convocar uma entrevista coletiva para d...,satire
1717,exclusivo veja a carta que temer escreveu para papai noel,alem da carta para a presidente dilma rousseff o vice presidente michel temer escreveu uma missi...,satire
38,liberacao de marcha da maconha pode provocar caos aereo por causa da fumaca,o brasil pode viver um caos aereo parecido com o que vivem os paises com vulcoes. a avaliacao e...,satire


----
# Análise de titúlos e textos  - parte 1 
 2 - Retornar os 3 vizinhos mais próximos usando os títulos como entrada e o conteúdo do texto das outras sátiras para o cálculo das similaridades;





In [ ]:
#nessa analise podemos descartar os label
#esse trecho de código também será utilizado na proxima analise 
df_titles_texts = df.drop(columns=['label'])

df_titles_texts.head()

,title,text
0,crise e tao grande que nem tiozao do pave fez piada na noite de natal,a familia guimaraes passou a noite de natal perplexa. genival o tiozao do pave deles desde 1985...
1,nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble,uma menina de 11 anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de j...
2,marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e...,insatisfeitas com seus partidos com as siglas dos partidos com os politicos com as tardes ensola...
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da america latina fez uma pesquisa que mostrou ideias que ...
4,apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz,assassinatos sequestros mortes violentas. nenhuma noticia parece abalar a apresentadora do cida...


In [ ]:
# Recuperando os textos da coluna title e text do dataframe
# esse trecho de código também será utilizado na proxima analise 
titles2 = df_titles_texts['title']
texts2 = df_titles_texts['text']


Fazendo os calculos:


In [ ]:
tfidf_matrix1, tfidf_vect1 = buildTFIDFVectorizer(texts2)


In [ ]:
query_doc2_1 = tfidf_matrix1.getrow(24)
query_doc2_2 = tfidf_matrix1.getrow(73)
query_doc2_3 = tfidf_matrix1.getrow(96)

In [ ]:
doc_sim2_1 = docSimilarities(query_doc2_1, tfidf_matrix1)
doc_sim2_2 = docSimilarities(query_doc2_2, tfidf_matrix1)
doc_sim2_3= docSimilarities(query_doc2_3, tfidf_matrix1)

In [ ]:
doc_sim2_1 = np.array(doc_sim2_1.toarray())
doc_sim2_2 = np.array(doc_sim2_2.toarray())
doc_sim2_3 = np.array(doc_sim2_3.toarray())

In [ ]:
mostSimilar2_1 = np.argsort(doc_sim2_1, axis=None)[-4:-1]
mostSimilar2_2 = np.argsort(doc_sim2_2, axis=None)[-4:-1]
mostSimilar2_3 = np.argsort(doc_sim2_3, axis=None)[-4:-1]


## Resultados: 


In [174]:
df.iloc[mostSimilar2_1]

,title,text,label
2113,cunhados farao trocadilhos com ovos 3. 457. 648 vezes nesta pascoa preve estudo,a cena e comum nas familias. o cunhado chega carregado na pascoa e antes de dar o presente diz ...,satire
421,98 das promessas de comer menos ovos na proxima pascoa nao serao cumpridas aponta estudo,poucas pessoas vao conseguir cumprir a promessa de comer menos ovos na proxima pascoa. um levan...,satire
1973,casal tira filho da escola para poder bancar ovo de pascoa neste ano,o casal evandro e herika matias de sao joao del rey mg prometeu ao pequeno mateus que neste ano ...,satire


In [179]:
df.iloc[mostSimilar2_2]

,title,text,label
2383,violencia vida real repete exatamente noticia do sensacionalista,e um ditado popular ladrao que rouba ladrao. . . mas e rarissimo no cotidiano. aconteceu ontem...,satire
3461,10 explicacoes para o buraco do maracana,de um helicoptero o reporter emerson rocha da radio globo tirou uma foto de uma cratera de quatr...,satire
3035,cunha e mais sete desculpas mirabolantes de politicos que nem o sensacionalista inventaria,eles sao muito muito criativos. basta que denuncias sejam feitas e investigacoes estejam em cur...,satire


In [181]:
df.iloc[mostSimilar2_3]

,title,text,label
3715,papai noel confirma que dilma nao sabia de nada sobre a petrobras,o bom velhinho estava cansado. ainda assim ele resolveu convocar uma entrevista coletiva para d...,satire
1936,papai noel tem problema no treno pega um uber e e linchado no aterro,uma tragedia se abateu sobre o natal dos cariocas. distribuindo os presentes dos moradores do r...,satire
3265,menino que pediu bicicleta e ganhou bola pede o impeachment de papai noel,nao e so a presidenta dilma rousseff que terminara o ano de 2015 sob risco de perder o cargo. p...,satire


----
# Análise de titúlos e textos  - parte 2
3 - Mesmo que 2 acima mas usando o corpo do texto também como entrada. Ou seja, usando o conteúdo textual de uma sátira como entrada e os outros conteúdos textuais para o cálculo das similaridades.

In [ ]:
tfidf_matrix2= tfidf_vect.transform(texts2)
tfidf_vect2 = buildTFIDFVectorizer(texts2)

In [ ]:
query_doc3_1 = tfidf_matrix2.getrow(24)
query_doc3_2 = tfidf_matrix2.getrow(73)
query_doc3_3 = tfidf_matrix2.getrow(96)

In [ ]:
doc_sim3_1 = docSimilarities(query_doc3_1, tfidf_matrix2)
doc_sim3_2 = docSimilarities(query_doc3_2, tfidf_matrix2)
doc_sim3_3= docSimilarities(query_doc3_3, tfidf_matrix2)

In [ ]:
doc_sim3_1 = np.array(doc_sim3_1.toarray())
doc_sim3_2 = np.array(doc_sim3_2.toarray())
doc_sim3_3 = np.array(doc_sim3_3.toarray())

In [ ]:
mostSimilar3_1 = np.argsort(doc_sim3_1, axis=None)[-4:-1]
mostSimilar3_2 = np.argsort(doc_sim3_2, axis=None)[-4:-1]
mostSimilar3_3 = np.argsort(doc_sim3_3, axis=None)[-4:-1]

## Resultados: 

In [175]:
df.iloc[mostSimilar3_1]

,title,text,label
2589,metade de ovo de pascoa na geladeira nao esta chamando seu nome a cada cinco minutos diz pisiqu...,voce sentado agora no sofa mexendo no celular ou no laptop esta com a impressao de ouvir uma voz...,satire
421,98 das promessas de comer menos ovos na proxima pascoa nao serao cumpridas aponta estudo,poucas pessoas vao conseguir cumprir a promessa de comer menos ovos na proxima pascoa. um levan...,satire
1973,casal tira filho da escola para poder bancar ovo de pascoa neste ano,o casal evandro e herika matias de sao joao del rey mg prometeu ao pequeno mateus que neste ano ...,satire


In [180]:
df.iloc[mostSimilar3_2]

,title,text,label
3461,10 explicacoes para o buraco do maracana,de um helicoptero o reporter emerson rocha da radio globo tirou uma foto de uma cratera de quatr...,satire
3410,14 fatos sobre michel temer que parecem coisa do sensacionalista,o senado esta para votar o afastamento da presidente dilma rousseff. se aprovado ela ficara pel...,satire
3035,cunha e mais sete desculpas mirabolantes de politicos que nem o sensacionalista inventaria,eles sao muito muito criativos. basta que denuncias sejam feitas e investigacoes estejam em cur...,satire


In [ ]:
df.iloc[mostSimilar3_3]

,title,text,label
3715,papai noel confirma que dilma nao sabia de nada sobre a petrobras,o bom velhinho estava cansado. ainda assim ele resolveu convocar uma entrevista coletiva para d...,satire
3265,menino que pediu bicicleta e ganhou bola pede o impeachment de papai noel,nao e so a presidenta dilma rousseff que terminara o ano de 2015 sob risco de perder o cargo. p...,satire
1936,papai noel tem problema no treno pega um uber e e linchado no aterro,uma tragedia se abateu sobre o natal dos cariocas. distribuindo os presentes dos moradores do r...,satire


----
# Respostas: 

**1 - Os vizinhos são os mesmos em todos os casos?**

 *Não, apesar de haver os mesmos vizinhos em comum nos casos, não são em todos os casos nos exemplos de textos escolhidos*


**2 - Se não, quais foram diferentes?**
*Nos 3 casos, há resultados diferentes, assim com resultados iguais, como por exemplo: no titulo 24, o primeiro caso resultou em " seca vaca de sao paulo produz leite em po", no segundo e terceiro caso não esteve, porém, mas em ambas "casal tira filho da escola para poder bancar ovo de pascoa neste ano" ocorreu como resultado, o mesmo acontece para os outros títulos escolhido como exemplo* 

**3 - Olhando a sátira de entrada, qual das configurações acima (1,2 ou 3) você acha que trouxe os melhores vizinhos e por quê?**

*A 2 e 3 tiveram resultado mais safistorio que o 1, pois 2 e 3 tem como calculo da matriz de TF-IDF o texto, com isso torna-se melhor que com a matriz sendo feita com o título*

**4 - Nos casos que houve diferença, por que você acha que isso aconteceu?**
*por ser apenas pesos de palavras e sua ocorrencia, nem sempre a ocorrência de uma palavra se dá em um mesmo contexto, podemos ter várias, e nos casos não foi diferente*
   
